# Pearson Chi-Square Test of Independence
*By P. Stikker*<br>
https://PeterStatistics.com<br>
https://www.youtube.com/stikpet<br>

## Introduction

To test if two nominal variables have an association, the most commonly used test is the Pearson chi-square test of independence. If the significance of this test is below 0.05, the two nominal variables have a significant association.

One problem though is that the Pearson chi-square test should only be used if not too many cells have a so-called expected count, of less than 5, and the minimum expected count is at least 1. So you will also have to check first if these conditions are met. Most often ‘not too many cells’ is fixed at no more than 20% of the cells. Note that there are othes who would say that all cells should have an expected count of at least 5.

Once you have checked the conditions and looked at the results, you can report the test results. In the example the percentage of cells with an expected count less than 5 is actually 0%, so it is okay to use the test.

Lets see how this works with Python by example.

## Example

The example data will be loaded as a Pandas Dataframe, so we'll need pandas:

In [ ]:
# !pip install pandas
import pandas as pd     # https://pandas.pydata.org/

To load the example data we can now use <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html">'read_csv'</a>. 

In [ ]:
myDf = pd.read_csv('../../Data/csv/GSS2012a.csv')
myDf.head()

I'll be using the 'mar1' and 'sex' fields for the example, so select those:

In [ ]:
myField1 = myDf['mar1']
myField2 = myDf['sex']

To get a quick look at the counts from these we can use Pandas '<a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html">crosstab</a>': 

In [ ]:
myCrosstable = pd.crosstab(myField1, myField2)
myCrosstable

To perform the test, I'll make use of the '<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html#scipy.stats.chi2_contingency">chi2_contingency</a>' function in the '<a href="https://docs.scipy.org/doc/scipy/reference/stats.html">stats</a>' module, of the <a href="https://www.scipy.org/">scipy</a> library. So lets load that in:

In [ ]:
# !pip install scipy
from scipy.stats import chi2_contingency

Now all we need to do is feed this function our cross table:

In [ ]:
chiVal, pVal, df, exp = chi2_contingency(myCrosstable)
chiVal, pVal, df, exp

The first value is the chi-square value of 16.99. The chance of such a value or even more extreme, in a sample, if there is no association in the population is 0.0019 (the second value). This is known as the p-value or significance. It is considered 'significant' usually if this value is below 0.05, which in this case it is. This indicates then an association between the two variables (one has an impact on the other).

The third value is the degrees of freedom, which is an indication of the size of the table, since it is simply the number of rows - 1, times the number of columns - 1.

The last array are the so called expected values. These are the counts to be expected if the two variables had no influence on each other.

Note that as a criteria the lowest expected value should be at least 1, and not too many should be below 5. Often a threshold of 20% is used (so if more than 20% of the cells have an expected count of 5 or less, the test is not reliable to use).

So lets check this. First the lowest expected count:

In [ ]:
exp.min()

So 35 is well above the criteria of at least 1, so thats good.

Now for the percentage of cells with a count less than 5:

In [ ]:
len(exp[exp < 5]) / len(exp) * 100

Well, since none of them was below 5 the percentage is of course 0%.

Instead of the scipy package, a slightly lesser known package of 'researchpy' also has a chi-square test that could be easier to use.

In [ ]:
# !pip install researchpy
import researchpy   # https://researchpy.readthedocs.io/

With this package we can use crosstab directly on the dataframe, set the test to 'chi-square', and since we're also interested in the expected frequencies, set 'expected_freqs' to True:

In [ ]:
crosstab, res, exp = researchpy.crosstab(myDf['sex'], myDf['mar1'], test='chi-square', expected_freqs=True)

The results of the test:

In [ ]:
res

And the expected values:

In [ ]:
exp

Same results again as before.

In the appendix I'll go over the formulas that are actually used in these packages.

## Appendix: The Hard Way

In [ ]:
myCrosstable

We begin with calculating the expected values. These can be calculated using:

\begin{equation*}
E_{i,j} = \frac{R_i \times C_j}{N}
\end{equation*}

The $E_{i,j}$ indicates the expected count in row i, column j. The $R_i$ is the row total of row i, and $C_j$ the column total of column j. The $N$ is the grand total.

So we need the column and row totals. This can be easily done with Pandas '<a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sum.html">sum</a>' function. For the rows we need to use 'axis=1' so it will sum by row (rather than by column):

In [ ]:
colTotals = myCrosstable.sum()
rowTotals = myCrosstable.sum(axis=1)

colTotals, rowTotals

Its useful to have the number of rows and columns also easily accessible, which can be obtained by using Python's '<a href="https://docs.python.org/3/library/functions.html#len">len</a>' function:

In [ ]:
nCols = len(colTotals)
nRows = len(rowTotals)

nCols, nRows

And the grand total is simply the sum of the row totals (or the column totals):

In [ ]:
N = rowTotals.sum()
N

Next we can determine the expected values using the formula. We do this for each cell so iterate over all rows and columns:

In [ ]:
for i in range(nRows):
    for j in range(nCols):
        E = rowTotals[i] * colTotals[j] / N
        print(E)

The chi-square value can now be calculated using:

\begin{equation*}
\chi^2 = \sum\frac{(O_{i,j}-E_{i,j})^2}{E_{i,j}}
\end{equation*}

The $O_{i,j}$ are the observed counts from the original cross table (i-th row, j-th column), we subtract the expected count, and square the result. We then add all of these up, and divide by the expected count.

So here goes with Python:

In [ ]:
chiVal = 0
for i in range(nRows):
    for j in range(nCols):
        E = rowTotals[i] * colTotals[j] / N
        O = myCrosstable.iloc[i,j]
        chiVal = chiVal + (O - E)**2 / E
chiVal

Great, the same result as with the scipy package.

We also will need the degrees of freedom (df), but that is relatively easy:

\begin{equation*}
df = (r - 1)(c-1)
\end{equation*}

Where $r$ is the number of rows, and $c$ the number of columns.

In [ ]:
df = (nRows - 1) * (nCols - 1)
df

To obtain the area under a chi-square distribution with df of 4, from 0 to our found chi-square value, we will use the chi2.cdf function from scipy.stats.

In [ ]:
# !pip install scipy
from scipy.stats import chi2

Now to get the area under the curve:

In [ ]:
chi2.cdf(chiVal, df)

However we actually need for the p-value a value equal or greater than the chi-square value. So we subtract the above result from 1:

In [ ]:
1 - chi2.cdf(chiVal, df)

And that's indeed the same as we saw before.

In [ ]:
chi2.sf(chiVal, df)